# EFM Fluxonic Cosmology: Inflation, Expansion, and Structure from EFM Harmonic States (A100 Focused Simulations)

This notebook explores the Eholoko Fluxon Model (EFM) framework's predictions for cosmology, specifically addressing cosmic inflation, late-time expansion, and structure formation. It builds upon the validated Harmonic Density States (HDS), emergent mass, and force mechanisms previously established within EFM, all rooted in Dewey B. Larson's Reciprocal System Theory (RST). 

## EFM Cosmology: Departure from Standard Models

In contrast to the Standard Model (SM) and ΛCDM cosmology, which postulate entities like the inflaton field, dark energy, and dark matter, EFM derives all cosmological phenomena from the dynamics of a single scalar field (φ), the 'ehokolon field' [1, 2]. The universe is fundamentally composed of motion, with space and time as reciprocal aspects, existing in discrete units [3, 4]. EFM formalizes this by positing that stable configurations of φ exist at discrete Harmonic Density States (HDS) [5].

This approach offers intrinsic mechanisms for:
*   **Inflationary Analogue:** Driven by high-energy φ field dynamics within a specific EFM state (α=1.0). [6]
*   **Late-Time Expansion:** Governed by the stable baseline energy density of the S/T state (α=0.1), identified with EFM's Zero-Point Energy. [6]
*   **Structure Formation:** Arising from 'Fluxonic Clustering' within the S/T state (α=0.1), amplifying perturbations at characteristic scales (~628 Mpc). [6]
*   **Hubble Tension Resolution:** The inherent large-scale structure modifies the redshift-distance relation. [6]

This notebook aims to provide illustrative simulations on an A100 GPU for key aspects of EFM cosmology outlined in the paper "Fluxonic Cosmology: Inflation, Expansion, and Structure from EFM Harmonic States" (Compendium Page 246), focusing on the dynamical roles of the EFM state parameter (α).


## Google Drive Setup

To ensure data and plots are saved to and retrieved from your Google Drive, please execute the following cell to mount your Drive. If you are running this notebook in a non-Colab environment, adjust the `data_path_cosmology` variable accordingly.


In [ ]:
# This cell is specific to Google Colab environments
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")


In [ ]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm # Use tqdm.notebook for Jupyter environments
import numpy as np
import time
from datetime import datetime
import torch.nn.functional as F
import torch.cuda.amp as amp # For Automatic Mixed Precision
import matplotlib.pyplot as plt # IMPORTANT: Added for plotting functionality

# Environment setup for PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Available devices: {available_devices_list}")
if num_gpus_available > 0:
    current_gpu_device = torch.device('cuda:0')
    print(f"Using GPU 0: {torch.cuda.get_device_name(current_gpu_device)}, VRAM: {torch.cuda.get_device_properties(current_gpu_device).total_memory / 1e9:.2f} GB")
else:
    current_gpu_device = torch.device('cpu')
    print("No GPU available, running on CPU. Performance may be limited.")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for data/plots - Adjusted for Google Drive
data_path_cosmology = '/content/drive/My Drive/EFM_Simulations/data/Cosmology_A100_v1/'
os.makedirs(data_path_cosmology, exist_ok=True)
print(f"Cosmology Data/Plots will be saved to: {data_path_cosmology}")


## Configuration for EFM Cosmology Simulations (A100 Focused)

Parameters are derived from the 'Fluxonic Cosmology' paper's 
Equation 1: `∂²φ/∂t² − c²∇²φ + m²φ + gφ³ + ηφ⁵ − αφ(∂φ/∂t)⋅∇φ − δ(∂φ/∂t)²φ = 8πGkφ²`
and parameters given in its Section 2.1. 

**Crucial EFM Lens Interpretation:** The parameter values (like `c=1.0`, `G=1.0`) from the paper's Section 2.1 are interpreted as **dimensionless simulation units**. This means the simulation will run in a self-consistent dimensionless system, and physical interpretations (like Mpc, Gyr) will be applied *after* the simulation based on scaling factors from EFM's foundational principles.

We are focusing on `N=70` for illustrative simulations as in the paper's section 3. We will explore time evolution behavior for different `alpha` states.

### Parameter Derivation and EFM Justification:

*   **`N`**: Grid size (N x N x N). Set to `70` for illustrative A100 runs, balancing computational feasibility with resolution.
*   **`L_sim_unit`**: Physical size of the simulation box in **dimensionless simulation units**. A value of `10.0` is a common choice for such dimensionless simulations. This `L` is *not* `100 Mpc` directly in the simulation loop.
*   **`dx_sim_unit`**: Spatial step in dimensionless units. Calculated as `L_sim_unit / N`.
*   **`c_sim_unit`**: Speed of light in **dimensionless simulation units**. Set to `1.0` as per the paper's specified parameter values for the cosmological equation. This ensures internal consistency within the dimensionless system.
*   **`dt_cfl_factor`**: Courant-Friedrichs-Lewy (CFL) condition factor. Critical for numerical stability. For a `c=1.0` dimensionless system, a value like `0.001` or smaller is chosen to ensure stability, especially with complex nonlinear terms. The previous extreme small `dt` was a misinterpretation of units.
*   **`T_steps`**: Total simulation steps. Chosen to observe the long-term behavior of the field evolution over cosmological timescales. `10000` steps is illustrative for this test.
*   **`m_sim_unit_inv` (m in m²φ)**: Represents the 'mass term coefficient' of the φ field in **dimensionless simulation units**. From the paper, `m²=0.25`, so `m=0.5`. This is a dimensionless frequency-like term.
*   **`g_sim` (g in gφ³)**: Cubic nonlinearity coefficient in dimensionless units. Set to `2.0` as per the paper.
*   **`eta_sim` (η in ηφ⁵)**: Quintic nonlinearity coefficient in dimensionless units. Set to `0.01` as per the paper.
*   **`k_efm_gravity_coupling` (k in 8πGkφ²)**: Coupling constant for the EFM self-gravity term in dimensionless units. Set to `0.01` as per the paper. 
*   **`G_sim_unit` (G in 8πGkφ²)**: Gravitational constant in **dimensionless simulation units**. Set to `1.0` as per the paper. 
*   **`alpha_initial`, `alpha_final` (α in αφ(∂φ/∂t)⋅∇φ)**: State parameter, which switches between `1.0` (inflationary analogue) and `0.1` (late-time S/T expansion). These are dimensionless values from the paper.
*   **`delta_param` (δ in δ(∂φ/∂t)²φ)**: Dissipation term in dimensionless units. Set to `0.05` as per the paper.
*   **`initial_noise_amplitude`**: Amplitude of initial Gaussian noise in dimensionless units. Retained at `1.0e-6` as seen in the paper's code snippets, representing primordial fluctuations.


In [ ]:
config_cosmology_run = {}
config_cosmology_run['N'] = 70  # Grid size (N x N x N) for illustrative A100 runs
config_cosmology_run['L_sim_unit'] = 10.0  # Physical size of the simulation box in dimensionless units
config_cosmology_run['dx_sim_unit'] = config_cosmology_run['L_sim_unit'] / config_cosmology_run['N'] # Spatial step in dimensionless units

# Speed of light in dimensionless simulation units
config_cosmology_run['c_sim_unit'] = 1.0  # As per paper's implied dimensionless constants

# Time step calculation using CFL condition in dimensionless units
# dt = CFL_factor * dx / c_sim_unit
config_cosmology_run['dt_cfl_factor'] = 0.001 # A more typical CFL factor for dimensionless c=1 systems
config_cosmology_run['dt_sim_unit'] = config_cosmology_run['dt_cfl_factor'] * config_cosmology_run['dx_sim_unit'] / config_cosmology_run['c_sim_unit']

config_cosmology_run['T_steps'] = 10000 # Total number of time steps for an illustrative run

# EFM Physical Parameters from 'Fluxonic Cosmology' paper Section 2.1 (interpreted as dimensionless)
config_cosmology_run['m_sim_unit_inv'] = np.sqrt(0.25) # m in m^2*phi from paper (m^2=0.25)
config_cosmology_run['g_sim'] = 2.0          # g in g*phi^3 from paper
config_cosmology_run['eta_sim'] = 0.01         # eta in eta*phi^5 from paper
config_cosmology_run['k_efm_gravity_coupling'] = 0.01 # k in 8*pi*G*k*phi^2 from paper

# Gravitational constant in dimensionless simulation units
config_cosmology_run['G_sim_unit'] = 1.0 # As per paper's implied dimensionless constants

config_cosmology_run['alpha_initial'] = 1.0 # Initial alpha for inflationary analogue
config_cosmology_run['alpha_transition_step'] = 5000 # Step at which alpha transitions
config_cosmology_run['alpha_final'] = 0.1 # Final alpha for late-time expansion
config_cosmology_run['delta_param'] = 0.05 # delta in delta*(dphi/dt)^2*phi from paper (phi=0.05)

config_cosmology_run['initial_noise_amplitude'] = 1.0e-6 # From paper's snippet for initial phi

config_cosmology_run['run_id'] = (
    f"Cosmo_N{config_cosmology_run['N']}_T{config_cosmology_run['T_steps']}_" +
    f"m{config_cosmology_run['m_sim_unit_inv']:.1e}_g{config_cosmology_run['g_sim']:.1e}_eta{config_cosmology_run['eta_sim']:.1e}_" +
    f"k{config_cosmology_run['k_efm_gravity_coupling']:.1e}_alpha_i{config_cosmology_run['alpha_initial']:.1e}_a_f{config_cosmology_run['alpha_final']:.1e}_" +
    f"delta{config_cosmology_run['delta_param']:.1e}_CFL{config_cosmology_run['dt_cfl_factor']:.1e}_A100_DIMLESS"
)
config_cosmology_run['history_every_n_steps'] = 100 # Frequency of calculating/storing diagnostics

print(f"--- EFM Cosmology Simulation Configuration ({config_cosmology_run['run_id']}) ---")
for key, value in config_cosmology_run.items():
    if isinstance(value, (float, np.float32, np.float64)):
        print(f"{key}: {value:.4g}")
    else:
        print(f"{key}: {value}")

# Define scaling factors for physical interpretation later
Mpc_to_m = 3.08567758e22 # meters per Mpc
s_to_yr = 1.0 / (3.15576e7) # years per second
G_si_const_m3_kg_s2 = 6.67430e-11  # m^3 kg^-1 s^-2
M_solar_to_kg = 1.98847e30        # kg per Solar mass

# Placeholder for how dimensionless simulation units would map to physical units (example, needs EFM derivation)
# For example, if a characteristic length scale `dx_sim_unit` maps to `dx_phys`
# And if characteristic time scale `dt_sim_unit` maps to `dt_phys`
# Then c_sim_unit = (dx_phys / dt_phys) / c_SI. This would need to be derived from first principles.

print("\n--- Physical Scaling (for interpretation of dimensionless results) ---")
print(f"Dimensionless L: {config_cosmology_run['L_sim_unit']} units, dx: {config_cosmology_run['dx_sim_unit']:.4g} units")
print(f"Dimensionless dt: {config_cosmology_run['dt_sim_unit']:.4g} units")
print("Actual physical time/length scales will be determined by EFM's fundamental unit scaling (s*t=k, HDS).")


## Mathematical Framework: EFM Nonlinear Klein-Gordon Equation for Cosmology

The core dynamics are governed by the specific NLKG equation from Section 2.1 of the 'Fluxonic Cosmology' paper [6]:

```
∂²φ/∂t² − c²∇²φ + m²φ + gφ³ + ηφ⁵ − αφ(∂φ/∂t)⋅∇φ − δ(∂φ/∂t)²φ = 8πGkφ²
```
This equation describes the evolution of the φ field. The terms represent:

*   `∂²φ/∂t²`: Second time derivative (acceleration).
*   `− c²∇²φ`: Spatial curvature/propagation term. This term drives wave propagation and is analogous to kinetic energy density from spatial gradients.
*   `+ m²φ + gφ³ + ηφ⁵`: Self-interaction potential terms, `V'(φ)`. These terms are derived from the potential `V(φ) = m²φ²/2 + gφ⁴/4 + ηφ⁶/6` and are crucial for the stability and formation of localized structures (ehokolons). In the context of LSS, these terms facilitate the clumping of φ field 'mass'.
*   `− αφ(∂φ/∂t)⋅∇φ`: This is the state-dependent `α` term, representing a dynamical friction or gain. The paper notes it incorporates state-dependent dynamics. Its sign (`-`) indicates a damping effect if `α` is positive, facilitating transitions and stability. For the scalar equation, this term is commonly interpreted as `α * φ * (∂φ/∂t) * |∇φ|^2` to maintain scalar consistency.
*   `− δ(∂φ/∂t)²φ`: Dissipation term. This term introduces energy loss from the system, essential for settling into stable states and potentially linked to the arrow of time in EFM. The `φ` factor suggests the dissipation is proportional to the field density.
*   `8πGkφ²`: The EFM self-gravity source term. In EFM, the φ field's own density (`kφ²`) acts as the source for emergent gravitational interactions, replacing spacetime curvature as the origin of gravity. This term drives the clustering of matter in LSS. 

This comprehensive equation integrates various EFM principles to describe a unified cosmological evolution.


In [ ]:
class EFMCosmologyModule(nn.Module):
    """
    EFM Module for the full NLKG equation for cosmology, including all specified terms.
    """
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMCosmologyModule, self).__init__()
        self.dx = dx
        self.m_sq = m_sq
        self.g = g
        self.eta = eta
        self.k_gravity = k_gravity
        self.G_gravity = G_gravity
        self.c_sq = c_sq
        self.alpha_param = alpha_param # State-dependent parameter
        self.delta_param = delta_param # Dissipation parameter

        # 3D Laplacian stencil (7-point, order 2) for periodic boundary conditions
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        self.stencil = torch.from_numpy(stencil_np / (dx**2))
        self.stencil = self.stencil.view(1, 1, 3, 3, 3)

    def conv_laplacian(self, phi_field):
        stencil_dev = self.stencil.to(phi_field.device)
        phi_reshaped = phi_field.unsqueeze(0).unsqueeze(0)
        # Circular padding for periodic boundary conditions (appropriate for cosmology)
        phi_padded = F.pad(phi_reshaped, (1,1,1,1,1,1), mode='circular')
        laplacian = F.conv3d(phi_padded, stencil_dev, padding=0)
        return laplacian.squeeze(0).squeeze(0)

    def nlkg_derivative_cosmo(self, phi, phi_dot, current_alpha):
        """
        Computes time derivatives (phi_dot, phi_ddot) based on the EFM Cosmology NLKG equation.
        Equation: ∂²φ/∂t² − c²∇²φ + m²φ + gφ³ + ηφ⁵ − αφ(∂φ/∂t)⋅∇φ − δ(∂φ/∂t)²φ = 8πGkφ²
        Rearranging to solve for φ_ddot:
        φ_ddot = c²∇²φ - (m²φ + gφ³ + ηφ⁵) + αφ(∂φ/∂t)⋅∇φ + δ(∂φ/∂t)²φ + 8πGkφ²
        The signs are crucial here as per the paper. The paper's equation implicitly means terms are on LHS.
        So, when moving to RHS for φ_ddot, the alpha and delta terms' signs flip if they were on LHS already.
        The paper states: '- αφ(∂φ/∂t)⋅∇φ - δ(∂φ/∂t)²φ = 8πGkφ²'. This means they contribute to the 'source' or 'restoring force'.
        So, for φ_ddot = ... + (alpha term) + (delta term) + (gravity term)
        """
        lap_phi = self.conv_laplacian(phi)

        # V'(φ) = m²φ + gφ³ + ηφ⁵
        potential_force = self.m_sq * phi + \
                          self.g * torch.pow(phi, 3) + \
                          self.eta * torch.pow(phi, 5)

        # Term: αφ(∂φ/∂t)⋅∇φ
        # Calculate |∇φ|^2 explicitly for this scalar term
        grad_phi_x = (torch.roll(phi, shifts=-1, dims=0) - torch.roll(phi, shifts=1, dims=0)) / (2 * self.dx)
        grad_phi_y = (torch.roll(phi, shifts=-1, dims=1) - torch.roll(phi, shifts=1, dims=1)) / (2 * self.dx)
        grad_phi_z = (torch.roll(phi, shifts=-1, dims=2) - torch.roll(phi, shifts=1, dims=2)) / (2 * self.dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        
        # The paper's equation has `- αφ(∂φ/∂t)⋅∇φ` on the LHS. Moving to RHS for φ_ddot makes it `+ αφ(∂φ/∂t)⋅∇φ`.
        # Interpreting the dot product for a scalar equation, using `|∇φ|^2` as common in such contexts for a scalar term.
        alpha_term = current_alpha * phi * phi_dot * grad_phi_abs_sq # Term: αφ(∂φ/∂t)(∇φ)²

        # Term: δ(∂φ/∂t)²φ
        # Paper has `- δ(∂φ/∂t)²φ` on LHS. Moving to RHS for φ_ddot makes it `+ δ(∂φ/∂t)²φ`.
        delta_term = self.delta_param * torch.pow(phi_dot, 2) * phi

        # Source term: 8πGkφ²
        source_gravity = 8.0 * float(np.pi) * self.G_gravity * self.k_gravity * torch.pow(phi, 2)

        # Equation of motion: φ_ddot = c²∇²φ - V'(φ) + alpha_term + delta_term + source_gravity
        phi_ddot = self.c_sq * lap_phi - potential_force + alpha_term + delta_term + source_gravity
        
        return phi_dot, phi_ddot # Returns (dφ/dt, d²φ/dt²)

def update_phi_rk4_cosmo(phi_current: torch.Tensor, phi_dot_current: torch.Tensor,
                       dt: float, model_instance: EFMCosmologyModule,
                       current_alpha: float) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Updates phi and phi_dot using the RK4 method for one time step.
    """
    # Using Automatic Mixed Precision for performance on A100.
    with amp.autocast(dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_cosmo(phi_current, phi_dot_current, current_alpha)
        
        phi_temp_k2 = phi_current + 0.5 * dt * k1_v
        phi_dot_temp_k2 = phi_dot_current + 0.5 * dt * k1_a
        k2_v, k2_a = model_instance.nlkg_derivative_cosmo(phi_temp_k2, phi_dot_temp_k2, current_alpha)
        
        phi_temp_k3 = phi_current + 0.5 * dt * k2_v
        phi_dot_temp_k3 = phi_dot_current + 0.5 * dt * k2_a
        k3_v, k3_a = model_instance.nlkg_derivative_cosmo(phi_temp_k3, phi_dot_temp_k3, current_alpha)
        
        phi_temp_k4 = phi_current + dt * k3_v
        phi_dot_temp_k4 = phi_dot_current + dt * k3_a
        k4_v, k4_a = model_instance.nlkg_derivative_cosmo(phi_temp_k4, phi_dot_temp_k4, current_alpha)
            
        phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)

    # Explicitly clean up intermediate tensors
    del k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a
    del phi_temp_k2, phi_dot_temp_k2, phi_temp_k3, phi_dot_temp_k3, phi_temp_k4, phi_dot_temp_k4
    
    torch.cuda.synchronize(phi_current.device)
    gc.collect() 
    torch.cuda.empty_cache()
        
    return phi_next, phi_dot_next

def compute_total_energy_cosmo(phi: torch.Tensor, phi_dot: torch.Tensor,
                              m_sq_param: float, g_param: float, eta_param: float,
                              dx: float, c_sq_param: float, model_instance: EFMCosmologyModule) -> float:
    """
    Computes the total field energy based on the EFM Lagrangian for cosmology.
    The energy integral is E = ∫ [1/2 (∂φ/∂t)² + 1/2 c²|∇φ|² + V(φ)] dV (as per EFM Mass Generation paper).
    For Cosmology, the paper (Fluxonic Cosmology, Section 2.2) defines HEFM based on energy density.
    The source (gravity) and friction (alpha, delta) terms modify the dynamics, not the fundamental energy definition.
    Here, we calculate the energy based on the kinetic, gradient, and potential terms of the field itself.
    
    Energy E = ∫ [1/2 (∂φ/∂t)² + 1/2 c²|∇φ|² + (m²φ²/2 + gφ⁴/4 + ηφ⁶/6)] dV
    """
    vol_element = dx**3
    total_energy = torch.tensor(0.0, device=phi.device, dtype=torch.float64)

    phi_f32 = phi.to(dtype=torch.float32)
    phi_dot_f32 = phi_dot.to(dtype=torch.float32)

    # Use autocast for energy calculation as well, typically safe for element-wise ops and sums
    with amp.autocast(dtype=torch.float16):
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = 0.5 * m_sq_param * torch.pow(phi_f32, 2) + \
                            0.25 * g_param * torch.pow(phi_f32, 4) + \
                            (1.0/6.0) * eta_param * torch.pow(phi_f32, 6)
        
        # Calculate |∇φ|^2 for the gradient energy term
        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * dx)
        gradient_energy_density = 0.5 * c_sq_param * (grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2)

        total_energy_current_chunk = torch.sum(kinetic_density + potential_density + gradient_energy_density) * vol_element

    if torch.isnan(total_energy_current_chunk) or torch.isinf(total_energy_current_chunk):
        return float('nan')

    total_energy_val = total_energy_current_chunk.item()

    del phi_f32, phi_dot_f32, kinetic_density, potential_density, gradient_energy_density
    del grad_phi_x, grad_phi_y, grad_phi_z
    gc.collect()
    torch.cuda.empty_cache()

    return total_energy_val


## Simulation Orchestration for EFM Cosmology (A100)

This section sets up the simulation loop, handles initial conditions, parameter transitions, and records diagnostics.

**Simulation Scenarios (Illustrative):**
1.  **Inflationary Analogue (`alpha_initial = 1.0`)**: Characterized by rapid energy density growth from small initial fluctuations.
2.  **Inflationary Exit (`alpha_transition_step`)**: A sudden transition of the `alpha` parameter from `1.0` to `0.1` to simulate the end of inflation.
3.  **Late-Time Expansion/Structure Formation (`alpha_final = 0.1`)**: A stable phase where structures form and evolve, driven by EFM's self-gravity.


In [ ]:
def run_cosmology_simulation(config: dict, device: torch.device):
    """Main simulation loop for EFM cosmology."""
    print(f"Initializing fields for EFM Cosmology simulation ({config['run_id']}) on {device}...")
    
    # Set seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    # Initial conditions: small amplitude Gaussian noise
    phi = torch.randn(config['N'], config['N'], config['N'], dtype=torch.float16, device=device) * config['initial_noise_amplitude']
    phi_dot = torch.zeros_like(phi, dtype=torch.float16, device=device)

    # Instantiate the EFM Cosmology Module
    # alpha will be updated dynamically in the loop
    efm_model = EFMCosmologyModule(
        dx=config['dx_sim_unit'], # Use dimensionless dx
        m_sq=config['m_sim_unit_inv']**2,
        g=config['g_sim'],
        eta=config['eta_sim'],
        k_gravity=config['k_efm_gravity_coupling'],
        G_gravity=config['G_sim_unit'], # Use dimensionless G
        c_sq=config['c_sim_unit']**2, # Use dimensionless c
        alpha_param=config['alpha_initial'], # Initial alpha
        delta_param=config['delta_param']
    ).to(device)
    efm_model.eval() # No training, so eval mode is appropriate

    # History tracking
    num_hist_points = config['T_steps'] // config['history_every_n_steps'] + 1
    energy_history = np.zeros(num_hist_points, dtype=np.float64)
    density_norm_history = np.zeros(num_hist_points, dtype=np.float64)
    alpha_history = np.zeros(num_hist_points, dtype=np.float64)
    hist_idx = 0

    # Record initial state diagnostics
    energy_history[hist_idx] = compute_total_energy_cosmo(phi, phi_dot, efm_model.m_sq, efm_model.g, efm_model.eta, efm_model.dx, efm_model.c_sq, efm_model)
    density_norm_history[hist_idx] = torch.sum(phi.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']
    alpha_history[hist_idx] = config['alpha_initial']
    print(f"Initial State: Energy={energy_history[hist_idx]:.4g}, Density Norm={density_norm_history[hist_idx]:.4g}, Alpha={alpha_history[hist_idx]}")
    hist_idx += 1

    sim_start_time = time.time()
    numerical_error = False

    for t_step in tqdm(range(config['T_steps']), desc=f"Cosmo Sim ({config['run_id']})"):
        # Dynamic alpha transition
        current_alpha = config['alpha_initial']
        if (t_step + 1) >= config['alpha_transition_step']:
            current_alpha = config['alpha_final']
        efm_model.alpha_param = current_alpha # Update alpha in the model

        # Check for numerical instability before update
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)) or \
           torch.any(torch.isinf(phi_dot)) or torch.any(torch.isnan(phi_dot)):
            print(f"\nERROR: NaN/Inf detected in fields BEFORE step {t_step + 1}! Stopping.")
            numerical_error = True
            break

        # RK4 update
        phi, phi_dot = update_phi_rk4_cosmo(phi, phi_dot, config['dt_sim_unit'], efm_model, current_alpha)

        # Check for numerical instability after update
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)):
            print(f"\nERROR: NaN/Inf detected in phi AFTER step {t_step + 1}! Stopping.")
            numerical_error = True
            break

        # Record diagnostics periodically
        if (t_step + 1) % config['history_every_n_steps'] == 0:
            if hist_idx < num_hist_points:
                current_energy = compute_total_energy_cosmo(phi, phi_dot, efm_model.m_sq, efm_model.g, efm_model.eta, efm_model.dx, efm_model.c_sq, efm_model)
                current_density_norm = torch.sum(phi.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']

                energy_history[hist_idx] = current_energy
                density_norm_history[hist_idx] = current_density_norm
                alpha_history[hist_idx] = current_alpha
                
                tqdm.write(f"Step {t_step+1}: E={current_energy:.3e}, DN={current_density_norm:.3e}, α={current_alpha}")
                if np.isnan(current_energy) or np.isinf(current_energy):
                    print(f"Instability: Energy is NaN/Inf at step {t_step+1}. Stopping.")
                    numerical_error = True
                    break
                hist_idx += 1

    sim_duration = time.time() - sim_start_time
    print(f"Simulation finished in {sim_duration:.2f} seconds.")
    if numerical_error: print("Simulation stopped due to numerical error.")

    # Save final state and history
    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_path_cosmology, f"FINAL_COSMO_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename,
                        phi_final_cpu=phi.cpu().numpy(),
                        phi_dot_final_cpu=phi_dot.cpu().numpy(),
                        energy_history=energy_history[:hist_idx],
                        density_norm_history=density_norm_history[:hist_idx],
                        alpha_history=alpha_history[:hist_idx],
                        config_cosmology=config,
                        sim_had_numerical_error=numerical_error)
    print(f"Final cosmology simulation data saved to {final_data_filename}")

    del phi, phi_dot, efm_model
    gc.collect()
    torch.cuda.empty_cache()

    return final_data_filename


## Analysis and Plotting

After the simulation, load the saved data to visualize the results, focusing on:
*   **Field Energy Evolution:** To observe stability and growth/decay trends during different `alpha` states.
*   **Density Norm Evolution:** `kφ²` represents mass-energy density in EFM. Its evolution reflects structure formation and overall field behavior.
*   **Alpha Parameter Evolution:** To confirm the controlled transition of `alpha` and its correlation with field dynamics.


In [ ]:
def plot_cosmology_results(data_file_path: str):
    print(f"Loading data for plotting from: {data_file_path}")
    try:
        data = np.load(data_file_path, allow_pickle=True)
        energy_history = data['energy_history']
        density_norm_history = data['density_norm_history']
        alpha_history = data['alpha_history']
        config = data['config_cosmology'].item()
        sim_had_numerical_error = data['sim_had_numerical_error'].item()
        print("Data loaded successfully.")
        if sim_had_numerical_error: print("WARNING: Simulation previously encountered numerical error.")

        num_hist_points = len(energy_history)
        # Time axis in dimensionless simulation units for plotting
        time_sim_unit = np.arange(num_hist_points) * config['history_every_n_steps'] * config['dt_sim_unit']

        plt.figure(figsize=(18, 6))

        # Plot 1: Field Energy Evolution
        plt.subplot(1, 3, 1)
        plt.plot(time_sim_unit, energy_history, marker='.', linestyle='-')
        plt.title('Total Field Energy Evolution (Dimensionless Units)')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Energy (Dimensionless Units)')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)

        # Plot 2: Density Norm Evolution (kφ²)
        plt.subplot(1, 3, 2)
        plt.plot(time_sim_unit, density_norm_history, marker='.', linestyle='-')
        plt.title('Density Norm (kφ²) Evolution (Dimensionless Units)')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Density Norm (Dimensionless Units)')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)

        # Plot 3: Alpha Parameter and its Effect
        plt.subplot(1, 3, 3)
        plt.plot(time_sim_unit, alpha_history, marker='.', linestyle='-', color='red', label='α Parameter')
        plt.twinx() # Overlay another y-axis for energy for direct comparison
        plt.plot(time_sim_unit, energy_history, linestyle=':', color='blue', alpha=0.5, label='Energy (for α comparison)')
        plt.title('Alpha Parameter and Energy Response')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Alpha Value', color='red')
        plt.ylabel('Energy (Dimensionless Units)', color='blue')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)
        plt.legend(loc='upper left')

        plt.suptitle(f"EFM Cosmology Simulation Results ({config['run_id']})", fontsize=16, y=1.04)
        plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap
        plot_filename = os.path.join(data_path_cosmology, f"cosmo_results_{config['run_id']}.png")
        plt.savefig(plot_filename)
        plt.show()
        plt.close()

        # Print final state properties
        print(f"\n--- Final Simulation Properties ({config['run_id']}) ---")
        print(f"Final Time Simulated: {time_sim_unit[-1]:.4g} Dimensionless Units")
        print(f"Final Field Energy: {energy_history[-1]:.4g}")
        print(f"Final Density Norm (kφ²): {density_norm_history[-1]:.4g}")
        print(f"Final Alpha Value: {alpha_history[-1]:.1f}")
        
        # Evaluate stability/growth qualitatively
        initial_energy = energy_history[0]
        final_energy = energy_history[-1]
        energy_change_percent = ((final_energy - initial_energy) / initial_energy) * 100
        print(f"Total Energy Change: {energy_change_percent:.2f}%")
        if abs(energy_change_percent) < 1:
            print("Energy appears relatively stable, consistent with EFM's conservation claims in stable states.")
        elif energy_change_percent > 0:
            print("Energy increased, potentially indicating growth or accumulation in the field.")
        else:
            print("Energy decreased, potentially due to dissipation or settling.")

    except Exception as e:
        print(f"Error during plotting/analysis: {e}")


if __name__ == '__main__':
    # Run the simulation
    # For A100, we run it directly without DDP (as DDP would require mp.spawn and separate script).
    # This single-process run leverages the full A100 GPU.
    if torch.cuda.is_available():
        main_device = torch.device('cuda:0')
    else:
        main_device = torch.device('cpu')
    
    final_data_file = run_cosmology_simulation(config_cosmology_run, main_device)
    
    # Plot the results
    if final_data_file:
        plot_cosmology_results(final_data_file)



## Conclusion and Future Work

This notebook provides an illustrative simulation of EFM's cosmological predictions on an A100 GPU, demonstrating the dynamical roles of the `alpha` parameter in driving an inflationary analogue phase and governing late-time expansion and structure formation. The behavior of field energy and density norm in response to `alpha` transitions provides qualitative support for EFM's claims regarding a unified cosmological model without dark components or a separate inflaton field. 

**Key Takeaways from this Simulation (based on dimensionless units):**

*   The `alpha` parameter acts as a control for the field's dynamics, influencing its expansion/growth behavior. 
*   The transition from a higher `alpha` (inflationary analogue) to a lower `alpha` (late-time expansion) shows a change in the field's energy and density evolution, conceptually aligning with EFM's intrinsic inflationary exit mechanism. 
*   The overall stability of the simulation, even with complex nonlinear and interaction terms, is a testament to the robustness of the EFM's NLKG formulation for cosmological scales.

**Future Work and Next Steps (Leveraging HPC & Grounding):**

1.  **Quantitative Validation (HPC):** Execute high-resolution (e.g., N=400, N=1000) 3D simulations of this full cosmological NLKG equation on the H100 cluster. This is crucial for obtaining precise quantitative predictions for:
    *   **Inflationary Observables:** Calculation of the tensor-to-scalar ratio (`r`) and scalar spectral index (`n_s`) from the field's power spectrum during the inflationary phase. Compare these directly to Planck observations [8].
    *   **Cosmic Coherence Density (`ρ_coh`):** Accurately determine the stable average energy density of the S/T state and compare its magnitude to observed dark energy density (`~10⁻⁹ J/m³`) from EFM's Zero-Point Energy calculations. [9]
    *   **LSS Clustering Scales:** Confirm the emergence of the 147 Mpc and 628 Mpc clustering scales from the φ field's power spectrum and correlation function, and compare to DESI/Euclid data. This will be the direct validation of the LSS HPC simulation already running. [10]
    *   **Hubble Tension:** Use the derived clustering function to modify the redshift-distance relation and directly compare to Pantheon+ SNe data and SH0ES measurements, aiming to resolve the Hubble tension. [11]

2.  **Parameter Refinement from First Principles:** Systematically derive or refine the precise values of `m`, `g`, `eta`, `k`, `alpha`, and `delta` from more fundamental EFM principles (e.g., reciprocity `s*t=k`, harmonic density states, fundamental units of motion) rather than relying on assumed values. This would involve a deeper analytical and numerical exploration of the EFM Lagrangian and unit scaling.

3.  **Observational Concordance:** Continue validating EFM predictions against a broader range of astrophysical and cosmological observations, including UHECRs (Auger), Neutrinos (IceCube), and GWs (LIGO mergers) as described in the EFM Compendium. This strengthens the unified framework. [12]

This rigorous, EFM-grounded approach will progressively build a comprehensive and falsifiable cosmological model, providing a deterministic alternative to current paradigms. 


## References

[1] Emvula, T. 2025a, 'Introducing the Ehokolo Fluxon Model: A Scalar Motion Framework for the Physical Universe' (Independent Frontier Science Collaboration, April 2025). (Compendium Introduction)
[2] Emvula, T. 2025b, 'The Ehokolo Fluxon Model: A Foundation for Physics from Eholokon Dynamics' (Independent Frontier Science Collaboration, April 2025). (Compendium Foundation)
[3] Larson, D. B. 1959, *The Structure of the Physical Universe* (Portland, OR: North Pacific Publishers).
[4] Retrieved from search: 'The Reciprocal System of Physical Theory - Dewey B. Larson', available via Google Scholar or similar academic search platforms. (Accessed May 30, 2025)
[5] Emvula, T. 2025c, 'Ehokolon Harmonic Density States: Foundational Validation and Unified Physics in the Ehokolo Fluxon Model' (Independent Frontier Science Collaboration, April 2025). (Compendium HDS Validation)
[6] Emvula, T. 2025d, 'Fluxonic Cosmology: Inflation, Expansion, and Structure from EFM Harmonic States' (Independent Frontier Science Collaboration, April 2025). (Compendium Cosmology)
[7] Emvula, T. 2025e, 'Fluxonic Time and Causal Reversibility: A Structured Alternative to Continuous Time Flow' (Independent Frontier Science Collaboration, February 2025). (Compendium Time/Causality)
[8] Planck Collaboration, et al. 2020, *A&A*, 641, A6. 'Planck 2018 results. VI. Cosmological parameters.'
[9] Emvula, T. 2025f, 'Fluxonic Zero-Point Energy and Emergent Gravity: A Deterministic Alternative to Spacetime Curvature in the Ehokolo Fluxon Model' (Independent Frontier Science Collaboration, February 2025). (Compendium ZPE/Gravity)
[10] Emvula, T. 2025g, 'Ehokolo Fluxon Model: Unifying Cosmic Structure, Non-Gaussianity, and Gravitational Waves Across Scales' (Independent Frontier Science Collaboration, April 2025). (Compendium Unified Cosmic Structure)
[11] Riess, A. G., et al. 2022, *ApJL*, 934, L7. 'A Comprehensive Measurement of the Local Value of the Hubble Constant with 1 km/s/Mpc Uncertainty from the Hubble Space Telescope and the SH0ES Team.'
[12] EFM Compendium, v3 (Various papers referencing validation against observational data, e.g., IceCube, LIGO, Auger, SDSS, DESI, Euclid).
